In [1]:
test_file = "1_cols_100_mb"

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LargeRowBenchMark").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/27 12:50:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/09/27 12:51:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Parquet

In [ ]:
df = spark.read.parquet(f"data/{test_file}.parquet")
df.createOrReplaceTempView("t")
df

In [ ]:
df.collect()

In [ ]:
from pyspark.sql.functions import array_max

df.select(array_max("col_0")).collect()

In [ ]:
from pyspark.sql.functions import array_sort

df.select(array_sort("col_0").alias('sorted_col_0')).collect()

In [ ]:
import pyspark.sql.functions as F

df.select(F.aggregate("col_0", F.lit(0.0), lambda acc, x: acc + x).alias("total")).collect()

In [ ]:
from pyspark.sql.types import DoubleType

def get_second_max(arr):
    if arr is None or len(arr) < 2:
        return None
    sorted_arr = sorted(arr, reverse=True)
    if len(sorted_arr) < 2:
        return None
    return sorted_arr[1]

spark.udf.register("get_second_max", get_second_max, DoubleType())
spark.sql("select get_second_max(col_0) from t").collect()

# CSV

In [ ]:
df = spark.read.csv(f"data/{test_file}.csv", header=True)
df

In [ ]:
# convert col_0 to an array of doubles
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import ArrayType, DoubleType

df = df.withColumn("col_0", from_json(col("col_0"), ArrayType(DoubleType())))
df.createOrReplaceTempView("t")
df

In [ ]:
df.collect()

In [ ]:
df.select(array_max("col_0")).collect()

In [ ]:
df.select(array_sort("col_0").alias('sorted_col_0')).collect()

In [ ]:
df.select(F.aggregate("col_0", F.lit(0.0), lambda acc, x: acc + x).alias("total")).collect()

In [ ]:
spark.sql("select get_second_max(col_0) from t").collect()

# Stop Spark

In [ ]:
spark.stop()